In [1325]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from fbprophet import Prophet
seaborn.set()

In [1326]:
comp_price = pd.read_csv('test_prc/comp_prices.csv', sep=',')
sales = pd.read_csv('test_prc/sales.csv', sep=',')

# B2W Challange

The sales.csv file contains transactional information where each line represents a sale. The comp_prices.csv file contains monitoring data of competitors’ prices. We have data available for 6 competitors, C1 to C6, which are monitored twice per day. The information below describes the data in each column:

- **PROD_ID**: Product ID. We provide data for 9 different products, P1 to P9;
- **DATE_ORDER**: Sales Date, under YYYY-MM-DD format;
- **QTY_ORDER*: Quantity Sold;
- **REVENUE**: Sale revenue. There can be variations of the price for the same product, depending on the sales channel or discounts, which are applied to the base price;
- **DATE_EXTRACTION**: Date and Time of the extraction of the competitors’ price, under YYYY-MM-DD HH:MM:SS format;
- **COMPETITOR**: Competitors’ ID (C1 to C6);
- **COMPETITOR_PRICE**: Competitors’ price per product, which can depend on the payment method;
- **PAY_TYPE**: Payment Method (1=deferred payment, 2=immediate payment).

1) Models for Demand Forecasting: The main objective is to create a model to predict the quantity sold for each product given a prescribed price. Along with the statistical model, we need metrics, relationships and descriptions of these data in order to understand the sales behavior. What does the data tell us? How are the different data sources related? Is there a particular competitor that seems more important?

2) Presentation of the results: we want to know what were the steps and your strategy (approach to the problem) during the analysis, even if these may seem wrong. The process you went through and the reasoning behind it, is as important as the solutions you found. For this, please prepare a clear and objective presentation to explain both your methodology and your results. In case you are selected for the interview, you will need to make a 20-minute (max) presentation.

In [1327]:
sales['DATE_ORDER'] = pd.to_datetime(sales['DATE_ORDER'])
comp_price['DATE_EXTRACTION_DAY'] = pd.to_datetime(comp_price['DATE_EXTRACTION'].str.split(' ').str[0])
comp_price['DATE_EXTRACTION'] = pd.to_datetime(comp_price['DATE_EXTRACTION'])

In [1328]:
sales

,PROD_ID,DATE_ORDER,QTY_ORDER,REVENUE
0,P6,2015-08-02,1.0,1808.99
1,P6,2015-08-17,1.0,1674.00
2,P6,2015-08-17,1.0,1673.95
3,P6,2015-08-11,1.0,1674.00
4,P6,2015-08-17,1.0,1674.00
...,...,...,...,...
351086,P3,2015-09-24,1.0,1008.83
351087,P3,2015-10-13,2.0,2333.92
351088,P3,2015-09-24,1.0,1311.81
351089,P3,2015-10-13,1.0,1166.96


In [1329]:
comp_price

,PROD_ID,DATE_EXTRACTION,COMPETITOR,COMPETITOR_PRICE,PAY_TYPE,DATE_EXTRACTION_DAY
0,P6,2015-01-30 08:11:38,C3,1898.00,1,2015-01-30
1,P6,2015-01-30 08:11:38,C1,1898.00,1,2015-01-30
2,P6,2015-01-30 08:11:38,C6,1999.90,1,2015-01-30
3,P6,2015-01-31 20:10:14,C2,1894.88,2,2015-01-31
4,P6,2015-01-31 20:10:14,C3,1894.88,2,2015-01-31
...,...,...,...,...,...,...
50109,P5,2015-10-11 20:10:34,C1,819.00,2,2015-10-11
50110,P5,2015-10-11 20:10:34,C2,853.52,2,2015-10-11
50111,P5,2015-10-12 08:11:27,C2,853.52,1,2015-10-12
50112,P5,2015-10-12 08:11:27,C3,819.00,1,2015-10-12


In [1330]:
len(sales)

351091

In [1331]:
len(comp_price)

50114

In [1332]:
comp_price2 = comp_price.groupby(['PROD_ID','DATE_EXTRACTION_DAY', 'COMPETITOR']).agg({'COMPETITOR_PRICE': ['min']}).reset_index()
comp_price2

,PROD_ID,DATE_EXTRACTION_DAY,COMPETITOR,COMPETITOR_PRICE
,,,,min
0,P1,2015-03-15,C5,1499.00
1,P1,2015-03-15,C6,1499.00
2,P1,2015-03-16,C2,1362.50
3,P1,2015-03-16,C5,1424.05
4,P1,2015-03-16,C6,1499.00
...,...,...,...,...
8655,P9,2015-10-14,C1,393.00
8656,P9,2015-10-14,C2,393.96
8657,P9,2015-10-14,C3,393.00


In [1333]:
comp_dummies = pd.get_dummies(comp_price2['COMPETITOR'])
comp_price2 = comp_price2.drop(['COMPETITOR'], axis=1)
comp_dummies

C:\Users\erycm\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,C1,C2,C3,C4,C5,C6
0,0,0,0,0,1,0
1,0,0,0,0,0,1
2,0,1,0,0,0,0
3,0,0,0,0,1,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
8655,1,0,0,0,0,0
8656,0,1,0,0,0,0
8657,0,0,1,0,0,0
8658,0,0,0,0,1,0


In [1334]:
dummies_arr = []
comp_pr = comp_price2['COMPETITOR_PRICE']['min'].array
i = 0 


for l in comp_dummies.to_numpy():
    arr = []
    for c in l:
        if c == 1:
            arr.append(comp_pr[i])
        else:
            arr.append(np.nan)
    i+=1
    dummies_arr.append(arr)

comp_dummies = pd.DataFrame(dummies_arr, columns=comp_dummies.columns)

comp_price = pd.concat([comp_price2, comp_dummies], axis=1)
comp_price.columns = ['PROD_ID', 'DATE_EXTRACTION_DAY', 'COMPETITOR_PRICE', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6']
comp_price

,PROD_ID,DATE_EXTRACTION_DAY,COMPETITOR_PRICE,C1,C2,C3,C4,C5,C6
0,P1,2015-03-15,1499.00,NaN,NaN,NaN,NaN,1499.00,NaN
1,P1,2015-03-15,1499.00,NaN,NaN,NaN,NaN,NaN,1499.00
2,P1,2015-03-16,1362.50,NaN,1362.50,NaN,NaN,NaN,NaN
3,P1,2015-03-16,1424.05,NaN,NaN,NaN,NaN,1424.05,NaN
4,P1,2015-03-16,1499.00,NaN,NaN,NaN,NaN,NaN,1499.00
...,...,...,...,...,...,...,...,...,...
8655,P9,2015-10-14,393.00,393.0,NaN,NaN,NaN,NaN,NaN
8656,P9,2015-10-14,393.96,NaN,393.96,NaN,NaN,NaN,NaN
8657,P9,2015-10-14,393.00,NaN,NaN,393.0,NaN,NaN,NaN
8658,P9,2015-10-14,465.52,NaN,NaN,NaN,NaN,465.52,NaN


In [1335]:
sales = sales.groupby(['PROD_ID','DATE_ORDER']).agg({'QTY_ORDER': ['sum'], 'REVENUE': ['min']}).reset_index()
sales


,PROD_ID,DATE_ORDER,QTY_ORDER,REVENUE
,,,sum,min
0,P1,2015-02-04,10.0,1499.00
1,P1,2015-02-05,12.0,1349.10
2,P1,2015-02-06,21.0,1349.10
3,P1,2015-02-07,4.0,1499.00
4,P1,2015-02-08,7.0,1499.00
...,...,...,...,...
2157,P9,2015-10-10,129.0,388.89
2158,P9,2015-10-11,87.0,388.66
2159,P9,2015-10-12,110.0,388.63


In [1336]:
len(sales)

2162

In [1337]:
len(comp_price)

8660

In [1338]:
df = pd.DataFrame.merge(sales, comp_price, how='left', left_on=['PROD_ID','DATE_ORDER'], right_on = ['PROD_ID','DATE_EXTRACTION_DAY'])
df.columns = ['PROD_ID', 'PROD_ID2', 'DATE_ORDER', 'QTY_ORDER', 'REVENUE', 'DATE_EXTRACTION_DAY', 'COMPETITOR_PRICE', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6']
df = df.drop(['PROD_ID2', 'DATE_EXTRACTION_DAY'], axis=1)

C:\Users\erycm\anaconda3\lib\site-packages\pandas\core\frame.py:9191: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


In [1339]:
df

,PROD_ID,DATE_ORDER,QTY_ORDER,REVENUE,COMPETITOR_PRICE,C1,C2,C3,C4,C5,C6
0,P1,2015-02-04,10.0,1499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P1,2015-02-05,12.0,1349.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,P1,2015-02-06,21.0,1349.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,P1,2015-02-07,4.0,1499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,P1,2015-02-08,7.0,1499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8853,P9,2015-10-14,590.0,341.1,393.00,393.0,NaN,NaN,NaN,NaN,NaN
8854,P9,2015-10-14,590.0,341.1,393.96,NaN,393.96,NaN,NaN,NaN,NaN
8855,P9,2015-10-14,590.0,341.1,393.00,NaN,NaN,393.0,NaN,NaN,NaN
8856,P9,2015-10-14,590.0,341.1,465.52,NaN,NaN,NaN,NaN,465.52,NaN
